In [25]:
%run run_voting_classifier.py x33t677d prw7kpf2 prw7kpf2 AttritionFlag unclassified -p ~/Documents/Classifiers/AthenaHealth/ -s

Loading Testing & Training documents...
No classifier found in /Users/kwang/Documents/Classifiers/AthenaHealth/.
Training classifier...
Testing classifier...
Test Accuracy:96.30%
